In [ ]:
import os
import tensorflow as tf
from preprocessing import inception_preprocessing
from nets import inception_v3

In [ ]:
def load_batch(dataset, batch_size, height, width, is_training=True):
    data_provider = slim.dataset_data_provider.DatasetDataProvider(dataset, common_queue_capacity=32, common_queue_min=8)
    image_raw, label = data_provider.get(['image','label'])
    image = inception_preprocessing.preprocess_image(image_raw, height, width, is_training=is_training)
    
    image_raw = tf.expand_dims(image_raw, 0)
    image_raw = tf.image.resize_images(image_raw, [height, width])
    image_raw = tf.squeeze(image_raw)
    
    images, images_raw, labels = tf.train.batch(
        [image, image_raw, label],
        batch_size=batch_size,
        num_threads=1,
        capacity=2*batch_size)

    return images, images_raw, labels
    

In [ ]:
def get_dataset(dataset_dir, train_sample_size, split_name):
    CLASS_NAMES=['t72','willys']
    file_pattern = os.path.join(dataset_dir, 't72willys_%s_*.tfrecord' % split_name)
    
    ITEMS_TO_DESCRIPTIONS = {
        'image' : 'images',
        'label' : 'labels'
    }
    
    keys_to_features = {
        'image/encoded' : tf.FixedLenFeature((), tf.string, default_value=''),
        'image/format' : tf.FixedLenFeature((), tf.string, default_value='png'),
        'image/class/label' : tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
    }
    
    items_to_handlers = {
        'image' : slim.tfexample_decoder.Image(),
        'label' : slim.tfexample_decoder.Tensor('image/class/label'),
    }
    
    labels_to_names = {}
    
    for i in range(0, len(CLASS_NAMES)):
        labels_to_names[i] = CLASS_NAMES[i]
        
    decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)
    
    return slim.dataset.Dataset(
        data_sources=file_pattern,
        reader = tf.TFRecordReader,
        decoder = decoder,
        num_samples = train_sample_size,
        items_to_descriptions = ITEMS_TO_DESCRIPTIONS,
        num_classes = len(CLASS_NAMES),
        labels_to_names=labels_to_names)

In [ ]:
def get_init_fn():
    """Returns a function run by the chief worker to warm-start the training."""
    checkpoint_exclude_scopes=["InceptionV3/Logits", "InceptionV3/AuxLogits", "InceptionV3/ConcatFC"]
    
    exclusions = [scope.strip() for scope in checkpoint_exclude_scopes]
    
    variables_to_restore = []
    variables_to_retrain = []
    

    for var in slim.get_model_variables():
        excluded = False
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
                break
        if not excluded:
            variables_to_restore.append(var)
        else:
            variables_to_retrain.append(var)
    
    with tf.gfile.Open('inceptionv3_restore_list.txt', 'w') as f:
        for var in variables_to_restore:
             f.write('%s ::RESTORED FROM CHECKPOINT\n' % (var))
        for var in variables_to_retrain:
             f.write('%s ::SELECTED FOR RETRAINING\n' % (var))
 
    
    return slim.assign_from_checkpoint_fn(
      '/root/qq/slim_test/ckpt/inception_v3.ckpt', variables_to_restore)

In [ ]:
TRAIN_SAMPLES = 500
INCEPTION_IMAGE_SIZE=299
BATCH_SIZE=32
NUMBER_OF_STEP=1500
LEARNING_RATE=0.0001
slim = tf.contrib.slim
TRAINED_MODEL_DIR='/root/qq/slim_test/training/zxcv'

In [ ]:
with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    train_dataset = get_dataset('/root/qq/t72_willys', TRAIN_SAMPLES, 'train')
    images, _, labels = load_batch(train_dataset, BATCH_SIZE, INCEPTION_IMAGE_SIZE, INCEPTION_IMAGE_SIZE)
    
    test_dataset = get_dataset('/root/qq/t72_willys', TRAIN_SAMPLES, 'validation')
    images2, _2, labels2 = load_batch(test_dataset, BATCH_SIZE, INCEPTION_IMAGE_SIZE, INCEPTION_IMAGE_SIZE)
    
    with slim.arg_scope(inception_v3.inception_v3_arg_scope()):
        net, _ = inception_v3.inception_v3(images, num_classes=None, is_training=True)
        tf.get_variable_scope().reuse_variables()
        net2, _2 = inception_v3.inception_v3(images2, num_classes=None, is_training=True)
        
        net = slim.dropout(net, keep_prob=0.8, scope='Dropout_1b')
        net2 = slim.dropout(net2, keep_prob=0.8, scope='Dropout_1b_2')
        _['PreLogits'] = net
        _2['PreLogits'] = net2
        
    with tf.variable_scope('InceptionV3/ConcatFC', reuse=tf.AUTO_REUSE) as scope:
        merged_net = tf.concat(axis=3, values=[net, net2], name='PreLogitsConcat')
        logits = slim.conv2d(merged_net, 2, [1,1], activation_fn=None, normalizer_fn=None, scope='Conv2d_1c_1x1')
        logits = tf.squeeze(logits, [1,2], name = 'SpatialSqueeze')
    
        one_hot_labels = slim.one_hot_encoding(labels, train_dataset.num_classes)
        slim.losses.softmax_cross_entropy(logits, one_hot_labels)
        total_loss = slim.losses.get_total_loss()
    
        tf.summary.scalar('losses/Total Loss', total_loss)
        optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE)
        train_op = slim.learning.create_train_op(total_loss, optimizer)
    
        final_loss = slim.learning.train(
            train_op,
            logdir=TRAINED_MODEL_DIR,
            init_fn=get_init_fn(),
            number_of_steps=NUMBER_OF_STEP)
        

In [ ]:
###evaluation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

BATCH_SIZE = 30
TEST_SAMPLE_SIZE = 1000

all_batch_stats = []

def getProbsAsStr(probabilities):
    CLASS_NAMES=['t72','willys']
    probs_str = 'Probabilities: '
    for label, prob in zip(CLASS_NAMES, probabilities):
        probs_str += '%s: %.2f%% ' % (label, prob*100)
    return probs_str

with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    train_dataset = get_dataset('/root/qq/t72_willys', TRAIN_SAMPLES, 'train')
    images, images_raw, labels = load_batch(
        train_dataset, BATCH_SIZE, INCEPTION_IMAGE_SIZE, INCEPTION_IMAGE_SIZE)
    
    test_dataset = get_dataset('/root/qq/t72_willys', TRAIN_SAMPLES, 'validation')
    images2, images_raw2, labels2 = load_batch(
        test_dataset, BATCH_SIZE, INCEPTION_IMAGE_SIZE, INCEPTION_IMAGE_SIZE)
    
    with slim.arg_scope(inception_v3.inception_v3_arg_scope()):
        net, _ = inception_v3.inception_v3(images, num_classes=None, is_training=True)
        tf.get_variable_scope().reuse_variables()
        net2, _2 = inception_v3.inception_v3(images2, num_classes=None, is_training=True)
        
        net = slim.dropout(net, keep_prob=0.8, scope='Dropout_1b')
        net2 = slim.dropout(net2, keep_prob=0.8, scope='Dropout_1b_2')
        _['PreLogits'] = net
        _2['PreLogits'] = net2
        
    with tf.variable_scope('InceptionV3/ConcatFC', reuse=tf.AUTO_REUSE) as scope:
        merged_net = tf.concat(axis=3, values=[net, net2], name='PreLogitsConcat')
        logits = slim.conv2d(merged_net, 2, [1,1], activation_fn=None, normalizer_fn=None, scope='Conv2d_1c_1x1')
        logits = tf.squeeze(logits, [1,2], name = 'SpatialSqueeze')

    probabilities = tf.nn.softmax(logits)
    
    checkpoint_path = tf.train.latest_checkpoint(TRAINED_MODEL_DIR)
    init_fn = slim.assign_from_checkpoint_fn(checkpoint_path, slim.get_variables_to_restore())
    
    with tf.Session() as sess:
        with slim.queues.QueueRunners(sess):
            sess.run(tf.initialize_local_variables())
            init_fn(sess)

            all_accuracy = []

            for i in range(int(TEST_SAMPLE_SIZE/BATCH_SIZE)):

                np_probabilities, np_images_raw, np_labels = sess.run([probabilities, images_raw, labels])
                
                all_batch_stats.append((np_labels, np_probabilities))
                
                if i == 1 or i == 5: # show images 
                    for j in range(BATCH_SIZE): 
                        image = np_images_raw[j, :, :, :]
                        true_label = np_labels[j]
                        predicted_label = np.argmax(np_probabilities[j, :])
                        predicted_name = test_dataset.labels_to_names[predicted_label]
                        true_name = test_dataset.labels_to_names[true_label]
                        plt.figure()
                        plt.imshow(image.astype(np.uint8))
                        plt.title('Ground Truth: [%s], Prediction [%s] '
                                  % (true_name, predicted_name) + getProbsAsStr(np_probabilities[j, :]))
                        plt.axis('off')
                        plt.show()

# Calculate accuracy over the whole test set
all_batch_accuracy = []
for labels, probs in all_batch_stats:
    for label, prob in zip(labels, probs):
            all_batch_accuracy.append(np.argmax(prob) == label)
print('Overall accuracy', np.mean(all_batch_accuracy))